In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/frames_zip/confusion_z.zip" -d "/content/drive/MyDrive/colab_data/"

Archive:  /content/drive/MyDrive/frames_zip/confusion_z.zip
  inflating: /content/drive/MyDrive/colab_data/Confusion/110001101243.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110001101255.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/11000120103.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110001201035.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110002103958.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110002204726.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110002204749.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110002204815.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110002204826.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110002204832.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110002204840.jpg  
  inflating: /content/drive/MyDrive/colab_data/Confusion/110002204857.jpg  
  inflating: /content/drive/M

In [ ]:
!unzip "/content/drive/MyDrive/frames_zip/Frustration_z.zip" -d "/content/drive/MyDrive/colab_data/"

In [ ]:
!unzip "/content/drive/MyDrive/frames_zip/Engagement_z.zip" -d "/content/drive/MyDrive/colab_data/"

In [ ]:
import os
import random 
import shutil
from os.path import exists

ratio=0.8
Emotions = ['Boredom', 'Engagement', 'Confusion', 'Frustration']
for emo in Emotions:
    files=os.listdir('/content/drive/MyDrive/colab_data/train/'+emo+'/')
    random.shuffle(files)
    target='/content/drive/MyDrive/colab_data/test/'+emo+'/'
    total_count=len(files)
    test_count = int(total_count * (1-ratio))
    train_count = total_count - test_count
    print(emo + '  total:' + str(total_count) + '  train:' + str(train_count) + '  test:' + str(test_count))
    for i in range(test_count):
        rand = random.randrange(1, total_count)
        target_file = target + files[rand]
        while exists(target_file):
            rand = random.randrange(1, total_count)
            target_file = target + files[rand]
        src='/content/drive/MyDrive/colab_data/train/'+emo+'/'+files[rand]
        shutil.move(src,target)
    print(emo + ' over')
print('Completed')



In [ ]:
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
train_path='/content/drive/MyDrive/colab_data/train/'
test_path='/content/drive/MyDrive/colab_data/test/'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['Boredom', 'Confusion', 'Engagement', 'Frustration']


In [ ]:
class ConvNet(nn.Module):
    
    def __init__(self,num_classes=4):
        
        super(ConvNet,self).__init__()

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
    def forward(self,input):
        
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
        output=output.view(-1,32*75*75)
            
        output=self.fc(output)
            
        return output

In [ ]:
model=ConvNet(num_classes=4).to(device)

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs = 10

In [ ]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [ ]:
print(train_count,test_count)

14459 3612


In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
      
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(5.1602) Train Accuracy: 0.5512829379625147 Test Accuracy: 0.5758582502768549
Epoch: 1 Train Loss: tensor(0.9480) Train Accuracy: 0.6492841828618854 Test Accuracy: 0.6868770764119602
Epoch: 2 Train Loss: tensor(0.7935) Train Accuracy: 0.6889134794937409 Test Accuracy: 0.7012735326688815
Epoch: 3 Train Loss: tensor(0.7373) Train Accuracy: 0.7012933121239366 Test Accuracy: 0.6990586932447398
Epoch: 4 Train Loss: tensor(0.6959) Train Accuracy: 0.7158862991908154 Test Accuracy: 0.7217607973421927
Epoch: 5 Train Loss: tensor(0.6624) Train Accuracy: 0.7239781450999377 Test Accuracy: 0.7112403100775194
Epoch: 6 Train Loss: tensor(0.6425) Train Accuracy: 0.7364271388062799 Test Accuracy: 0.6932447397563677
Epoch: 7 Train Loss: tensor(0.6149) Train Accuracy: 0.7479770385227195 Test Accuracy: 0.721483942414175
Epoch: 8 Train Loss: tensor(0.6028) Train Accuracy: 0.7508126426447196 Test Accuracy: 0.7037652270210409
Epoch: 9 Train Loss: tensor(0.5821) Train Accuracy: 0.75